This notebook is part of [miubrt](https://github.com/meteo-ubonn/miubrt).

Copyright (c) [miubrt developers](https://github.com/meteo-ubonn/miubrt/blob/main/CONTRIBUTORS.txt). 
Distributed under the MIT License. See [LICENSE.txt](https://github.com/meteo-ubonn/miubrt/blob/main/LICENSE.txt) for more info.

# Create NetCDF4 output - Explore JuXPol
<div class="alert alert-info">

**User Info** <br>
 Global Data - User Input 
 
 start_time, end_time, location and sweep

 location = "BoXPol" or "JuXPol"  (X-band radar)
 
 elev_id  = 0 ... 9   (IDs for elevation angles)
    
 For BoXPol, the elevation folders name change in 2017 (check for error message)
   
</div>
 

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import datetime as dt
import numpy as np
import dask
import wradlib as wrl
import xarray as xr

In [ ]:
# for using local development version, uncomment next lines otherwise installed `miubrt` version will be used
# miubrt_dir = "../../"
# if sys.path[0] != miubrt_dir:
#     sys.path.insert(0, "../../")

In [ ]:
import miubrt as mrt
mrt.version.version

# Global Data - USER INPUT

In [ ]:
start_time = dt.datetime(2015, 7, 5, 14, 0)
end_time = dt.datetime(2015, 7, 5, 14, 59, 59)
location = "JuXPol"
elev_id  = 0

In [ ]:
if location == "BoXPol":
    if start_time < dt.datetime(2017, 4, 16):
        snames = ["n_ppi_010deg","ppi_1p5deg","ppi_2p4deg","ppi_3p4deg","n_ppi_045deg","n_ppi_082deg", 
              "n_ppi_110deg","n_ppi_140deg","n_ppi_180deg","n_ppi_280deg"]
    else:
        snames = ["n_ppi_010deg","n_ppi_020deg","n_ppi_031deg","n_ppi_045deg","n_ppi_060deg","n_ppi_082deg", 
                  "n_ppi_110deg","n_ppi_140deg", "n_ppi_180deg", "n_ppi_280deg"]
    sname  = snames[elev_id]
    sweep  = 0
elif location == "JuXPol":
    sname = "DWD_Vol_2"
    sweep = elev_id

In [ ]:
output_path = ""

# Create Netcdf Dataset of wanted radar data

In [ ]:
fname_raw = mrt.io.create_netcdf_dataset(location, 
                                         sname, 
                                         start_time, 
                                         end_time, 
                                         sweep, 
                                         outpath=output_path, 
                                         engine='h5netcdf', 
                                         chunks=None)
fname_raw

# Load and Explore Dataset

In [ ]:
import matplotlib as mpl
import hvplot
import hvplot.xarray
import holoviews as hv

## Load

In [ ]:
swp = xr.open_dataset(fname_raw, chunks={"time": 1})

In [ ]:
swp = swp.assign_coords(elevation=swp.elevation.median("time"))

## Georeference

In [ ]:
swp = swp.pipe(wrl.georef.georeference_dataset)

## Interactive Exploration

In [ ]:
swp = swp.chunk()

In [ ]:
dbz_plot = swp.hvplot.quadmesh(groupby='time',
                               x='x', y='y', 
                               z='DBZH', 
                               rasterize=True, 
                               clim=(-20,50), 
                               cmap="turbo",
                               frame_width=500,
                               aspect=1,
                              )

dbz_plot = dbz_plot.options(color_levels=list(np.arange(-20, 51, 2)))
dbz_plot